# Generación de Data Única referente a Duraciones

Transformación de algunos datos para normalizar nombres en columnas

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from unicodedata import normalize

from src.data import cleandata

In [2]:
tqdm.pandas()

path_interim = "../data/interim/pjud"
path_processed = "../data/processed/pjud"

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# DATA de TRIBUNALES OBTENIDAS DEL COT

In [3]:
# Analizo contra los datos extraidos en COT
df_tribunales = pd.read_feather(f"{path_processed}/processes_ListadoTribunales.feather")
tribunales = df_tribunales['TRIBUNAL'].unique()

# DATA DURACION


In [4]:
df_duracion = pd.read_feather(f"{path_interim}/clean_Duraciones.feather")
df_duracion.sort_values('FECHA INGRESO')

,level_0,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,FECHA TERMINO,DURACIÓN CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS
1076293,1076405,401124,56,C.A. DE PUERTO MONTT,662,JUZGADO DE LETRAS Y GARANTIA DE QUELLON,684-2018,Ordinaria,1900-01-01,2018-08-21,43331,SENTENCIA,ago-18,2018,1
1013095,1013202,337911,50,C.A. DE TEMUCO,214,JUZGADO DE LETRAS Y GARANTIA DE PUCON,1059-2017,Ordinaria,1900-01-01,2018-10-01,43372,DECLARA SOBRESEIMIENTO DEFINITIVO.,oct-18,2018,1
975151,975257,299951,46,C.A. DE CONCEPCION,1080,JUZGADO DE GARANTIA DE TALCAHUANO,2398-2017,Ordinaria,1900-01-01,2018-05-31,43249,DECLARA SOBRESEIMIENTO DEFINITIVO.,may-18,2018,1
2767837,2768176,91242,25,C.A. DE LA SERENA,932,JUZGADO DE GARANTIA DE OVALLE,1-2000,Ordinaria,2000-12-20,2015-06-02,5277,CERTIFICA CUMPLIMIENTO ART. 468.,jun-15,2015,1
2770323,2770662,93728,25,C.A. DE LA SERENA,932,JUZGADO DE GARANTIA DE OVALLE,3-2000,Ordinaria,2000-12-21,2015-06-02,5276,NO PERSEVERAR EN EL PROCEDIMIENTO.,jun-15,2015,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649335,649398,649466,91,C.A. DE SAN MIGUEL,1237,JUZGADO DE GARANTIA DE SAN BERNARDO,15092-2019,Ordinaria,2019-12-31,2019-12-31,0,APROBACION NO INICIO INVESTIGACION.,dic-19,2019,1
649334,649397,649465,91,C.A. DE SAN MIGUEL,1237,JUZGADO DE GARANTIA DE SAN BERNARDO,15091-2019,Ordinaria,2019-12-31,2019-12-31,0,APROBACION NO INICIO INVESTIGACION.,dic-19,2019,1
552119,552162,552230,90,C.A. DE SANTIAGO,1228,9º JUZGADO DE GARANTIA DE SANTIAGO,13267-2019,Ordinaria,2019-12-31,2019-12-31,0,APROBACION NO INICIO INVESTIGACION.,dic-19,2019,1
526568,526604,526672,90,C.A. DE SANTIAGO,1226,7º JUZGADO DE GARANTIA DE SANTIAGO,22351-2019,Ordinaria,2019-12-31,2019-12-31,0,COMUNICA Y/O APLICA DECISION PPIO. DE OPORTUNI...,dic-19,2019,1


In [5]:
filtro_fecha = df_duracion[df_duracion['FECHA INGRESO']<='2014-12-31']
print(f"Existen {len(filtro_fecha)} causas ingresadas antes del 01-01-2015")

Existen 410874 causas ingresadas antes del 01-01-2015


In [6]:
df_duracion.drop(filtro_fecha.index, axis=0, inplace=True)

In [7]:
filtro_null = df_duracion[df_duracion['FECHA INGRESO'].isnull()]
print(f"Existen {len(filtro_null)} causas sin fecha de ingresos")

Existen 0 causas sin fecha de ingresos


In [8]:
df_duracion.drop(filtro_null.index, axis=0, inplace=True)

In [9]:
# Normalizo nombres

df_duracion['TRIBUNAL'] = df_duracion['TRIBUNAL'].progress_apply(cleandata.cambio_nombre_juzgados)

100%|██████████| 2945536/2945536 [00:16<00:00, 175322.84it/s]


In [10]:
# Verifico si coinciden los nombres e imprimo los que estan en conflicto

for tribunal in df_duracion['TRIBUNAL'].unique():
    if not tribunal in tribunales:
        print(tribunal)

## CARGA DATA A ARCHIVOS FUTHER

In [12]:
# Directorio donde se guardaran archivos feather
df_duracion.reset_index(inplace=True, drop=True)
os.makedirs(path_processed, exist_ok = True)  
# Guardamos dataset como archivo feather
df_duracion.to_feather(f'{path_processed}/processes_Duraciones.feather')